From the RFC guided example in the Bootcamp 3.2.5.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

First do the cleaning as outlined in the guided example.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

y2015 = pd.read_csv('LoanStats3d_1.csv', skipinitialspace=True, header=1)

# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

y2015 = y2015[:-2]

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


array([0.97858042, 0.9805514 , 0.98083636, 0.98171499, 0.97183567,
       0.9790786 , 0.92702273, 0.98047923, 0.98016957, 0.9800266 ])

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [2]:
#Dropping anything related to payment amount or outstanding principal.
y2015.drop(y2015.filter(regex = 'pymnt_amnt').columns, axis = 1, inplace = True)
y2015.drop(y2015.filter(regex = 'bal').columns, axis = 1, inplace = True)

#Dropping useless columns by visual checking
y2015.drop(['policy_code', 'id', 'member_id', 'title'], 1, inplace=True)

#Correlation matrix
cor = y2015.corr()

#Dropping highly correlated columns
y2015.drop(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'out_prncp_inv',
                    'total_pymnt_inv', 'total_rec_prncp', 'collection_recovery_fee'], 1, inplace=True)

#Redo the RFC
X_new = y2015.drop('loan_status', 1)
Y_new = y2015['loan_status']
X_new = pd.get_dummies(y2015)
X_new = X_new.dropna(axis=1)

cross_val_score(rfc, X_new, Y_new, cv=10)

array([1.        , 0.99995251, 0.99997625, 0.99995251, 0.99997625,
       1.        , 0.9999525 , 1.        , 0.9999525 , 0.99997625])

In [5]:
X_new.head()

,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,out_prncp,...,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,14.85,379.39,48000.0,33.18,0.0,0.0,11.0,2.0,19.0,13668.88,...,0,0,0,0,0,1,0,0,0,0
1,7.49,298.58,60000.0,22.44,0.0,0.0,7.0,0.0,9.0,6635.69,...,0,0,0,0,0,1,0,0,0,0
2,7.49,777.55,109000.0,26.02,0.0,1.0,9.0,0.0,19.0,0.00,...,0,0,0,0,0,1,0,0,0,0
3,6.49,858.05,92000.0,21.60,0.0,0.0,16.0,0.0,24.0,19263.77,...,0,0,0,0,0,1,0,0,0,0
4,19.89,320.99,55000.0,25.49,0.0,4.0,18.0,1.0,19.0,0.00,...,0,0,0,0,0,1,0,0,0,0


### PCA


In [6]:
X_pca = StandardScaler().fit_transform(X_new)
pca = PCA(n_components=165)
Y_pca = pca.fit_transform(X_pca)

pca.explained_variance_ratio_

array([4.61853765e-02, 3.54851423e-02, 2.52083320e-02, 2.29805393e-02,
       1.83766256e-02, 1.76016374e-02, 1.38187527e-02, 1.32581412e-02,
       1.29117216e-02, 1.22900628e-02, 1.11052316e-02, 1.08014645e-02,
       1.04987280e-02, 1.02729130e-02, 9.90990855e-03, 9.78846491e-03,
       9.42658539e-03, 9.20472397e-03, 8.95402794e-03, 8.87488608e-03,
       8.68109339e-03, 8.53583418e-03, 8.40196648e-03, 8.38050992e-03,
       8.30308045e-03, 8.21350750e-03, 8.16706891e-03, 8.12027835e-03,
       8.04597222e-03, 7.95435243e-03, 7.87975755e-03, 7.83893546e-03,
       7.74039531e-03, 7.61817073e-03, 7.55171165e-03, 7.47772007e-03,
       7.45647298e-03, 7.41516247e-03, 7.39581202e-03, 7.28470268e-03,
       7.20574209e-03, 7.13538092e-03, 7.09686059e-03, 7.05166207e-03,
       7.00073272e-03, 6.91411947e-03, 6.88126921e-03, 6.79651296e-03,
       6.75896400e-03, 6.72495341e-03, 6.68230962e-03, 6.66707273e-03,
       6.61682834e-03, 6.61311032e-03, 6.58420918e-03, 6.54442406e-03,
      

Looks like the first 18 components at least explain much more than the rest. So I will just take the first 18 out.

In [7]:
df_components = pd.DataFrame(Y_pca)
pca_features = df_components.iloc[:, : 18]

X_new_pca = pca_features
Y_new_pca = y2015['loan_status']

cross_val_score(rfc, X_new_pca, Y_new_pca, cv=10)

array([0.99280473, 0.99313719, 0.99325592, 0.99199734, 0.99206839,
       0.99280456, 0.9927094 , 0.99282814, 0.99199658, 0.99021517])